In [1]:
import numpy as np
import math

In [3]:
def tauh(cfl, u, c):
    return cfl / max(max(abs(u-c)), max(abs(u+c)))

In [4]:
def first_phase(H_n_mid, u_n_mid, H_n, u_n, g, tau_h, N):
    H_centre = np.zeros(N, np.double) #переменные в цетнре ячеек
    u_centre = np.zeros(N, np.double)
    
    for i in range(N):
        H_centre[i] = H_n_mid[i] - tau_h * (H_n[i+1] * u_n[i+1] - H_n[i] * u_n[i]) / 2
        u_centre[i] = (H_n_mid[i] * u_n_mid[i] - tau_h * (H_n[i+1] * u_n[i+1] ** 2 - H_n[i] * u_n[i] ** 2) / 2 -
                      g * tau_h * (H_n[i+1] ** 2 - H_n[i] ** 2) / 4) / H_centre[i]

    return (H_centre, u_centre)

In [5]:
def second_phase(H_n, u_n, g, H_centre, u_centre, H_n_mid, u_n_mid, N, u_left, u_right):
    
    R_n = np.array([u_n[i] + 2 * math.sqrt(g * H_n[i]) for i in range(N+1)], dtype = np.double)
    Q_n = np.array([u_n[i] - 2 * math.sqrt(g * H_n[i]) for i in range(N+1)], dtype = np.double)
    R_centre = np.array([u_centre[i] + 2 * math.sqrt(g * H_centre[i]) for i in range(N)], dtype = np.double)
    Q_centre = np.array([u_centre[i] - 2 * math.sqrt(g * H_centre[i]) for i in range(N)], dtype = np.double)
    
    R_n_mid = np.array([u_n_mid[i] + 2 * math.sqrt(g * H_n_mid[i]) for i in range(N)], dtype = np.double)
    Q_n_mid = np.array([u_n_mid[i] - 2 * math.sqrt(g * H_n_mid[i]) for i in range(N)], dtype = np.double)
    
    R_n1 = np.zeros(N+1, np.double)
    Q_n1 = np.zeros(N+1, np.double)
    
    for i in range(N): #вычисление инвариантов + коррекция
        
        minR = min(R_n[i], R_n_mid[i], R_n[i+1])
        maxR = max(R_n[i], R_n_mid[i], R_n[i+1])
            
        if u_centre[i] + math.sqrt(g * H_centre[i]) > 0:
            R_n1[i+1] = 2 * R_centre[i] - R_n[i]
            
            if R_n1[i+1] < minR:
                R_n1[i+1] = minR
            if R_n1[i+1] > maxR:
                R_n1[i+1] = maxR
                
        else:
            R_n1[i] = 2 * R_centre[i] - R_n[i+1]
            
            if R_n1[i] < minR:
                R_n1[i] = minR
            if R_n1[i] > maxR:
                R_n1[i] = maxR
                
        minQ = min(Q_n[i], Q_n_mid[i], Q_n[i+1])
        maxQ = max(Q_n[i], Q_n_mid[i], Q_n[i+1])        
        
        if u_centre[i] - math.sqrt(g * H_centre[i]) > 0:
            Q_n1[i+1] = 2 * Q_centre[i] - Q_n[i]
            
            if Q_n1[i+1] < minQ:
                Q_n1[i+1] = minQ
            if Q_n1[i+1] > maxQ:
                Q_n1[i+1] = maxQ
        else:
            Q_n1[i] = 2 * Q_centre[i] - Q_n[i+1]
            
            if Q_n1[i] < minQ:
                Q_n1[i] = minQ
            if Q_n1[i] > maxQ:
                Q_n1[i] = maxQ
                
                
    for i in range(N-1): #коррекция для звуковых точек
        if (u_centre[i] + math.sqrt(g * H_centre[i]) > 0 and u_centre[i+1] + math.sqrt(g * H_centre[i+1]) < 0) or \
        (u_centre[i] + math.sqrt(g * H_centre[i]) < 0 and u_centre[i+1] + math.sqrt(g * H_centre[i+1]) > 0):
            R_n1[i+1] = (R_centre[i] + R_centre[i+1]) / 2
            
        if (u_centre[i] - math.sqrt(g * H_centre[i]) > 0 and u_centre[i+1] - math.sqrt(g * H_centre[i+1]) < 0) or \
        (u_centre[i] - math.sqrt(g * H_centre[i]) < 0 and u_centre[i+1] - math.sqrt(g * H_centre[i+1]) > 0):
            Q_n1[i+1] = (Q_centre[i] + Q_centre[i+1]) / 2
                
    
                 
    u_n1 = np.zeros(N+1, np.double) #переменные нового слоя
    H_n1 = np.zeros(N+1, np.double)
    
    u_n1[0] = u_left
    u_n1[N] = u_right
    H_n1[0] = math.pow(Q_n1[0] - u_left, 2) / (4 * g)
    H_n1[N] = math.pow(R_n1[N] - u_right, 2) / (4 * g)
    
    for i in range(1, N):
        u_n1[i] = (R_n1[i] + Q_n1[i]) / 2
        H_n1[i] = ((R_n1[i] - Q_n1[i]) ** 2) / 16 / g
        
    return (H_n1, u_n1)

In [6]:
def third_phase(H_n1, u_n1, H_centre, u_centre, g, tau_h, N):
    H_n1_mid = np.zeros(N, dtype = np.double) #переменные середин отрезков нового слоя
    u_n1_mid = np.zeros(N, dtype = np.double)
    
    for i in range(N):
        H_n1_mid[i] = H_centre[i] - tau_h * (H_n1[i+1] * u_n1[i+1] - H_n1[i] * u_n1[i]) / 2
        u_n1_mid[i] = (H_centre[i] * u_centre[i] - tau_h * (H_n1[i+1] * u_n1[i+1] ** 2 - H_n1[i] * u_n1[i] ** 2) / 2 -
                      g * tau_h * (H_n1[i+1] ** 2 - H_n1[i] ** 2) / 4) / H_n1_mid[i]
        
    return (H_n1_mid, u_n1_mid)

#### Задаем начальные условия и сетку

In [21]:
N = 100
a = 0
b = 10
grid = np.linspace(a, b, N+1)

cfl = 0.1
g = 1

u_left = 0
u_right = 0
H_left = 2
H_right = 1

u0 = np.array([u_left for _ in range(N // 2)] + [u_right for _ in range(N // 2 + 1)], dtype = np.double)
H0 = np.array([H_left for _ in range(N // 2)] + [H_right for _ in range(N // 2 + 1)], dtype = np.double)

In [12]:
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [22]:
H_pre = H0
u_pre = u0
H_n_mid = np.array([(H0[i] + H0[i+1]) / 2 for i in range(N)], dtype = np.double)
u_n_mid = np.array([(u0[i] + u0[i+1]) / 2 for i in range(N)], dtype = np.double)

In [23]:
for i in range(1000):
    
    
        
    c = np.array([math.sqrt(g * H_pre[i]) for i in range(N+1)])
    
    tau_h = tauh(cfl, u_pre, c)
    
    H_centre, u_centre = first_phase(H_n_mid, u_n_mid, H_pre, u_pre, g, tau_h, N)
    
    H_next, u_next = second_phase(H_pre, u_pre, g, H_centre, u_centre, H_n_mid, u_n_mid, N, u_left, u_right)
    
    H_n_mid, u_n_mid = third_phase(H_next, u_next, H_centre, u_centre, g, tau_h, N)
    
    H_pre = H_next
    u_pre = u_next
    
    clear_output(True)
    plt.axis([a,b,0,3])
    plt.plot(grid, H_pre, color = 'red', linewidth=1)
    plt.show()

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>